In [ ]:
! pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 1.7 MB/s eta 0:00:00


### Load libraries

In [ ]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torcheval.metrics.functional import (
    multiclass_accuracy, multiclass_auprc
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%load_ext autoreload
%autoreload 2
sys.path.append('/content/drive/MyDrive/Deep_learning_course/Lung_segmentation/classification_problem/')
from custom_load_resnet import LungDatasetClassif
from data_loading_resnet import load_train_data, load_test_data, load_train_eval_data
from fit_func_resnet_dual import FitTrainEval
from resnet18_dual_model import ResNet18_dual

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Move data to local path

In [ ]:
path_to_zip_folder = '/content/drive/MyDrive/Deep_learning_course/Lung_segmentation/data/m3ex02-data.zip'
current_folder_path = '/content/localpath/'
!unzip -q $path_to_zip_folder -d $current_folder_path

### Parameters

In [ ]:
work_dir: str = "/content/drive/MyDrive/Deep_learning_course/Lung_segmentation/classification_problem/"
# data parameters
data_path: str = "/content/localpath/"
folder_name: str = "m3ex02-data"
train_path: str = os.path.join(data_path, folder_name, "Train/")
val_path: str = os.path.join(data_path, folder_name, "Val/")
test_path: str = os.path.join(data_path, folder_name, "Test/")
# workers: int = 12
# batch_size: int = 800
workers: int = 8
batch_size: int = 350

# model parameters
metrics: dict = [multiclass_accuracy, multiclass_auprc]
model_folder: str = "models"
model_name: str = "covid_classif_masks_11.pkl"
model_name_load: str = "covid_classif_masks_9.pkl"
save_path: str = os.path.join(work_dir, model_folder, model_name)
in_channels: int = 1
classes: int = 3
seg_channels: int = 1
EPOCHS: int = 100

### Import data

In [ ]:
data_train = load_train_data(
    train_path,
    batch_size=batch_size,
    workers=workers,
    pin_memory_device=device
)
data_val = load_train_data(
    val_path,
    batch_size=batch_size,
    workers=workers,
    pin_memory_device=device
)
# data_train = load_train_data(
#     test_path,
#     batch_size=batch_size,
#     workers=workers,
#     pin_memory_device=device
# )

In [ ]:
for i in range(1):
  data = next(iter(data_train))
  print(data["mask"].shape)

torch.Size([350, 1, 224, 224])


In [ ]:
# test_data = LungDatasetClassif(
#     root=val_path,
# )

In [ ]:
# test_data[0:3]["image"].shape

In [ ]:
# data_train, data_val = load_train_eval_data(
#     path_to_data=val_path,
#     batch_size=batch_size,
#     eval_size=0.2,
#     workers=workers,
#     pin_memory_device=device
# )

In [ ]:
len(data_train), len(data_val)

(63, 16)

In [ ]:
# data["image"][0].shape

In [ ]:
# for i in range(1):
#   data = next(iter(data_train))
#   print(data["image"][0].shape)

In [ ]:
# data_train.pin_memory_device, data_val.pin_memory_device

### Model

In [ ]:
# model = ResNetCovid(in_channels, classes)
# model = model.to(device)

In [ ]:
# model = ResNet(img_channels=in_channels, num_classes=classes, layers=18, block=BlockModel18_34)
# model = model.to(device)

In [ ]:
model = ResNet18_dual(img_channels=in_channels, num_classes=classes)
state = torch.load(os.path.join(work_dir, model_folder, model_name_load))
model.load_state_dict(state["model"])
model = model.to(device)

In [ ]:
next(model.parameters()).is_cuda

True

In [ ]:
weights = torch.tensor([0.25, 0.05, 0.7], dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=5e-7)
# optimizer.load_state_dict(state["optimizer"])

In [ ]:
optimization = FitTrainEval(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    metrics=metrics,
    path_save=save_path,
    device=device
)

In [ ]:
trained = optimization.fit_train_eval(
    data_train=data_train,
    data_eval=data_val,
    epochs=EPOCHS,
)

Epoch 1/100


  0%|          | 0/63 [00:00<?, ?it/s]

Evaluating


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 1.0577015772698417 	 Val loss: 1.0707523487508297
Train multiclass_accuracy: 0.4254119374922344 	 Val multiclass_accuracy: 0.44532185373827815
Train multiclass_auprc: 0.33568967050976223 	 Val multiclass_auprc: 0.3445751219987869

Best metric: -inf
Current metric: 0.44532185373827815
Epoch time: 57.00040626525879 s
Saving model
Epoch 2/100


...

Evaluating


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 0.9796078063192821 	 Val loss: 1.0609745346009731
Train multiclass_accuracy: 0.7596976477948446 	 Val multiclass_accuracy: 0.4495551730506122
Train multiclass_auprc: 0.3361110994740138 	 Val multiclass_auprc: 0.34808775782585144

Best metric: 0.49800470285117626
Current metric: 0.4495551730506122
Epoch time: 53.49845623970032 s
Epoch 20/100


  0%|          | 0/63 [00:00<?, ?it/s]

KeyboardInterrupt: 